In [ ]:
import cv2
import numpy as np

# Exercise 1

Frequency Domain

Spatial Domain

Average Mean Squared Error

In [ ]:
def amse(A, B):
    error = np.sum((A.astype("float") - B.astype("float")) ** 2)
    error /= float(A.shape[0] * A.shape[1])
    return error

Results for Exercise 1

# Exercise 2

No Reference IQA

Results for Exercise 2